In [1]:
pip install dataset tqdm pandas


     ---------------------------------------- 1.6/1.6 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 210.4/210.4 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB 4.3 MB/s eta 0:00:00


In [2]:
pip install datasets


     ------------------------------------ 452.9/452.9 kB 726.9 kB/s eta 0:00:00
     ------------------------------------ 110.5/110.5 kB 802.2 kB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 4.0 MB/s eta 0:00:00
     -------------------------------------- 139.5/139.5 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 323.5/323.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB 3.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [5]:
#Loading dataset
dataset = load_dataset('tapaco', 'en')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/158053 [00:00<?, ? examples/s]

Dataset tapaco downloaded and prepared to C:/Users/HP/.cache/huggingface/datasets/tapaco/en/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paraphrase_set_id', 'sentence_id', 'paraphrase', 'lists', 'tags', 'language'],
        num_rows: 158053
    })
})

In [7]:
dataset.keys()

dict_keys(['train'])

In [21]:
dataset['train'][0].keys()

dict_keys(['paraphrase_set_id', 'sentence_id', 'paraphrase', 'lists', 'tags', 'language'])

In [26]:
dataset['train'][0]['paraphrase']

'I ate the cheese.'

In [27]:
# conting loaded data and saveing into local repo

In [28]:
def process_tapaco_dataset(dataset, out_file):
    tapaco = []
    # The dataset has only train split.
    for data in tqdm(dataset["train"]):
        keys = data.keys()
        tapaco.append([data[key] for key in keys])
    tapaco_df = pd.DataFrame(
        data=tapaco,
        columns=[
            "language",
            "lists",
            "paraphrase",
            "paraphrase_set_id",
            "sentence_id",
            "tags",
        ],
    )
    tapaco_df.to_csv(out_file, sep="\t", index=None)
    return tapaco_df

tapaco_df = process_tapaco_dataset(dataset,"tapaco_huggingface.csv")
tapaco_df.head()

100%|████████████████████████████████████████████████████████████████████████| 158053/158053 [00:17<00:00, 8814.51it/s]


,language,lists,paraphrase,paraphrase_set_id,sentence_id,tags
0,1,416554,I ate the cheese.,"[907, 4000, 6677, 7361, 7415]",[],en
1,1,2481696,I eat cheese.,[907],[],en
2,1,2721028,I'm eating a yogurt.,"[992, 3800]",[],en
3,1,3010891,I'm eating cheese.,[6905],[],en
4,1,4129977,I'm having some cheese.,[6905],[],en


In [29]:
tapaco_df.shape

(158053, 6)

In [72]:
def generate_tapaco_paraphrase_dataset(dataset, out_file):
    dataset_df = dataset[["paraphrase", "paraphrase_set_id"]]
    non_single_labels = (
        dataset_df["paraphrase_set_id"]
        .value_counts()[dataset_df["paraphrase_set_id"].value_counts() > 1]
        .index.tolist()
    )
    tapaco_df_sorted = dataset_df.loc[
        dataset_df["paraphrase_set_id"].isin(non_single_labels)
    ]
    tapaco_paraphrases_dataset = []

    for paraphrase_set_id in tqdm(tapaco_df_sorted["paraphrase_set_id"].unique()):
        id_wise_paraphrases = tapaco_df_sorted[
            tapaco_df_sorted["paraphrase_set_id"] == paraphrase_set_id
        ]
        len_id_wise_paraphrases = (
            id_wise_paraphrases.shape[0]
            if id_wise_paraphrases.shape[0] % 2 == 0
            else id_wise_paraphrases.shape[0] - 1
        )
        for ix in range(0, len_id_wise_paraphrases, 2):
            current_phrase = id_wise_paraphrases.iloc[ix][0]
            for count_ix in range(ix + 1, ix + 2):
                next_phrase = id_wise_paraphrases.iloc[ix + 1][0]
                tapaco_paraphrases_dataset.append([current_phrase, next_phrase])
    tapaco_paraphrases_dataset_df = pd.DataFrame(
        tapaco_paraphrases_dataset, columns=["Text", "Paraphrase"]
    )
    tapaco_paraphrases_dataset_df.to_csv(out_file, sep="\t", index=None)
    return tapaco_paraphrases_dataset_df

tapaco_paraphrases_dataset_df = generate_tapaco_paraphrase_dataset(tapaco_df,"tapaco_paraphrases_dataset.csv")

1                              [907]
3                             [6905]
4                             [6905]
6                  [907, 4000, 7415]
8      [907, 4000, 7360, 7389, 7409]
                   ...              
495                [907, 4000, 7412]
496                [907, 4000, 7412]
497                [907, 4000, 7412]
498                            [956]
499                           [6905]
Name: paraphrase_set_id, Length: 392, dtype: object


In [1]:
import pandas as pd
dataset_df = pd.read_csv("tapaco_paraphrases_dataset.csv",sep="\t")
dataset_df.head()

,Text,Paraphrase
0,I ate the cheese.,I eat cheese.
1,I'm eating a yogurt.,I'm eating cheese.
2,I'm having some cheese.,I eat some cheese.
3,It's Monday.,It is Monday today.
4,It's Monday today.,Today is Monday.


In [2]:
# Renaming the columns
dataset_df.columns = ["input_text","target_text"]
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
dataset_df["prefix"] = "paraphrase"
dataset_df.head()

,input_text,target_text,prefix
0,I ate the cheese.,I eat cheese.,paraphrase
1,I'm eating a yogurt.,I'm eating cheese.,paraphrase
2,I'm having some cheese.,I eat some cheese.,paraphrase
3,It's Monday.,It is Monday today.,paraphrase
4,It's Monday today.,Today is Monday.,paraphrase


In [3]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(dataset_df,test_size=0.1)

In [4]:
train_data

,input_text,target_text,prefix
6766,I'll call at your house tomorrow.,Goodbye. See you tomorrow.,paraphrase
15902,Maybe next time.,Some other time perhaps.,paraphrase
56334,He often adopts the behaviours and speech patt...,He often adopts the behaviors and speech patte...,paraphrase
57850,They look worried.,They seem worried.,paraphrase
44766,The surgery went well.,The operation was successful.,paraphrase
...,...,...,...
1902,You need to exercise more.,You must study harder.,paraphrase
25048,Tom climbed out the window.,Tom crawled out of the window.,paraphrase
62587,Let's check with him.,Let's check with her.,paraphrase
21537,Tom is a native French speaker.,French is Tom's primary language.,paraphrase


In [5]:
test_data

,input_text,target_text,prefix
43568,Your ideas are rather old-fashioned.,His ideas are a little archaic.,paraphrase
47880,Tom told me he was going to talk to Mary.,Tom told me he'd talk to Mary.,paraphrase
200,Today is a warm day.,Today is a very hot day.,paraphrase
34324,She felt faint at the sight of blood.,She fainted when she saw blood.,paraphrase
2201,"Fill out this form, please.","Will you fill out this form, please?",paraphrase
...,...,...,...
25438,They drink coke.,They drink cola.,paraphrase
59363,Tom and I were both invited.,Both Tom and I were invited.,paraphrase
10241,Do you want fruit juice?,Do you want some fruit juice?,paraphrase
72752,They say that they're sleepy.,They say they're sleepy.,paraphrase


In [25]:
# pip install simpletransformers
# pip install torch
# pip install transformers[torch]==4.
from simpletransformers.t5 import T5Model
from tensorflow import keras

args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    "adam_epsilon": 1e-08,
    "eval_batch_size": 6,
    "fp_16": False,
    "gradient_accumulation_steps": 16,
    "learning_rate": 0.0003,
    "max_grad_norm": 1.0,
    "n_gpu": 1,
    "seed": 42,
}

In [22]:
model = T5Model("t5","t5-small", args=args, use_cuda=False)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [29]:
model.save_model()

In [27]:
import sklearn
model.train_model(train_data, eval_data=test_data, use_cuda=False,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/66166 [00:00<?, ?it/s]

c:\users\hp\appdata\local\programs\python\python39\lib\site-packages\transformers\tokenization_utils_base.py:3478: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/8271 [00:00<?, ?it/s]


KeyboardInterrupt



In [34]:
from simpletransformers.t5 import T5Model
import os
root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"")

In [42]:
from simpletransformers.t5 import T5Model
import os
# trained_model_path = os.path.join(root_dir,"outputs")
args = {
"overwrite_output_dir": True,
"max_seq_length": 256,
"max_length": 50,
"top_k": 50,
"top_p": 0.95,
"num_return_sequences": 5
}
trained_model_path =  'MODEL/checkpoint-2064-epoch-4'
trained_model = T5Model("t5",trained_model_path,args=args, use_cuda=False)

In [40]:
import warnings
warnings.filterwarnings('ignore')


prefix = "paraphrase"
pred = trained_model.predict([f"{prefix}: In the African folktales, the stories reflect the culture where diverse types of animals abound"])

print('Paraphrased')
print(pred[0])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/5 [00:00<?, ?it/s]

Paraphrased
['In the African folktales, the stories reflect the culture where diverse types of animals abound.', 'In African folktales, stories reveal the culture where a lot of animals can survive.', 'In Africa, stories are a symbol of culture where diversity of animals exists.', 'These stories reflect the culture of where diverse types of animals exist.', 'In the African folktales, stories reflect culture in which many species of animals abound abound; and there exists a lot of cultural diversity in the region.']


In [41]:
for p in pred[0]: 
    print(p)

In the African folktales, the stories reflect the culture where diverse types of animals abound.
In African folktales, stories reveal the culture where a lot of animals can survive.
In Africa, stories are a symbol of culture where diversity of animals exists.
These stories reflect the culture of where diverse types of animals exist.
In the African folktales, stories reflect culture in which many species of animals abound abound; and there exists a lot of cultural diversity in the region.
